In [109]:
import sys
sys.path.append('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/backend/chatbot_utils')

import context_model as cm
import importlib
importlib.reload(cm)

import pandas as pd
import hashlib
import json


with open('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/openAI_api.json') as f:
    key = json.load(f)

### Testing Embedding and Context Models

In [9]:
DATA_DIR = 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset'
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()

In [6]:
cm.convert_logs_to_embeddings(f"{DATA_DIR}/test_log1.out")

In [13]:
returns = cm.get_context("When was run sfa_mmi executed?", f"./chromadb/{md5hash}_large")
returns

([Document(page_content='Nov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:63]: sri_recall_log_lookup_location: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:69]: working_dir: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.__init__:43]: SRSS version: 05.01.01\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [utilities.register_in_persistence_registry_service:215]: Successfully registered in PersistenceRegistryService\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [srss.__init__:51]: Service registered\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27255]: PyInstaller _MEI directory save_rec

### ChatGPT test

In [110]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

import os
os.environ["OPENAI_API_KEY"] = key['openai_api_key']

In [111]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, input_key='question', output_key='answer')

In [112]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()
question = "Can you give me a summary of the log file? Start with SUMMARY:"

In [113]:
docs_small,conf_small = cm.get_context(question, f"./chromadb/{md5hash}_small")
docs_large,conf_large = cm.get_context(question, f"./chromadb/{md5hash}_large")

## Add both large and small to a single list and sort them by confidence
docs = docs_small + docs_large
conf = conf_small + conf_large

df = pd.DataFrame({'docs':docs, 'conf':conf})
df.sort_values(by='conf', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# df



In [114]:
list(df['docs'].values)

[Document(page_content='Nov 09 13:13:26 CMX50070-101776 start_mrt.sh[16216]:     TextLogMaxSize:                 1 MB\nNov 09 13:13:26 CMX50070-101776 start_mrt.sh[16216]:     TextLogMaxNumFiles:             10\nNov 09 13:13:26 CMX50070-101776 start_mrt.sh[16216]:     MaxNumEventsBacked:             1000', metadata={'source': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out'}),
 Document(page_content='Nov 09 19:17:08 CMX50070-101776 start-mrt.sysmond.sh[2352936]: 2023-11-09T19:17:08,557366079+00:00 Compress log "/home/instrument/fw/log/sysmond_main.1.nmon"...\nNov 09 19:17:08 CMX50070-101776 start-mrt.sysmond.sh[2352938]: *** zstd command line interface 64-bits v1.5.2, by Yann Collet ***\nNov 09 19:17:08 CMX50070-101776 start-mrt.sysmond.sh[2352938]: [295B blob data]\nNov 09 19:17:08 CMX50070-101776 start-mrt.sysmond.sh[2352945]: 2023-11-09T19:17:08,718960566+00:00 Compression done\nNov 09 19:17:08 CMX50070-101776 start-mrt.sysmond.sh[2352960]: 2023-11-

In [115]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

vectorstore_relevant = Chroma.from_documents(list(df['docs'].values), embedding=embedding)

In [116]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)

In [117]:
retriver = vectorstore_relevant.as_retriever()
chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True)

In [118]:
result = chat({"question": question})
result

{'question': 'Can you give me a summary of the log file? Start with SUMMARY:',
 'chat_history': [HumanMessage(content='Can you give me a summary of the log file? Start with SUMMARY:'),
  AIMessage(content='SUMMARY: The log file details the start of the start-mrt.sysmond.sh script on the system CMX50070-101776. It begins with the compression of the log file "/home/instrument/fw/log/sysmond_main.1.nmon" using the zstd command line interface 64-bits v1.5.2. It then compresses another log file "/home/instrument/fw/log/sysmond_processes.1.nmon" with the same command line interface. Both compressions are completed. The log file also indicates that the maximum size for a text log is 1 MB, the maximum number of text log files is 10, and the maximum number of events that can be backed is 1000.\n')],
 'answer': 'SUMMARY: The log file details the start of the start-mrt.sysmond.sh script on the system CMX50070-101776. It begins with the compression of the log file "/home/instrument/fw/log/sysmond_

### Summarization Test

In [56]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

# Loading the datastore
vector_db = Chroma(persist_directory=f"./chromadb/{md5hash}_large", embedding_function=embedding)
retriver = vector_db.as_retriever()

In [57]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)

chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True)

In [58]:
result = chat({"question": f"Give me a long summary of the document. Start with SUMMARY:"})
result

{'question': 'Give me a long summary of the document. Start with SUMMARY:',
 'chat_history': [HumanMessage(content='Give me a long summary of the document. Start with SUMMARY:'),
  AIMessage(content='SUMMARY: The document does not provide enough information to generate a long summary.\n')],
 'answer': 'SUMMARY: The document does not provide enough information to generate a long summary.\n',
 'sources': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out',
 'source_documents': [Document(page_content='Nov 10 05:49:10 CMX50070-101776 audit: PROCTITLE proctitle=2F7573722F7362696E2F69707461626C6573002D2D77616974002D740066696C746572002D4400444F434B45520000002D690062722D653937343130663832613539002D6F0062722D653937343130663832613539002D7000746370002D64003130302E38322E36342E313330002D2D64706F72740034393935002D6A004143\nNov 10 05:49:10 CMX50070-101776 audit[2208785]: NETFILTER_CFG table=nat family=2 entries=18 op=xt_replace pid=2208785 comm="iptables"\nNov 10 05:49:

In [46]:
edit = "Add some information about the Publish update"
result = chat({"question": f"Edit the previous summary. {edit}. Start with 'SUMMARY:' followed by the edited version of the previous summary."})
result

{'question': "Edit the previous summary. Add some information about the Publish update. Start with 'SUMMARY:' followed by the edited version of the previous summary.",
 'chat_history': [HumanMessage(content='Give me a summary. Start with SUMMARY:'),
  AIMessage(content='SUMMARY: The document provides details about various processes starting within a certain application at around 13:15:10 on November 9, 2023. These processes include the WCDMA State Servicer and GSM State Servicer, and the use of persistent settings and a pull_url. Additionally, the log entries detail the initialization of a system identified as CMX50070-101776, including the initialization of logging with level 20, the DriftRpcClient with a specific URL, a message dispatcher, and a default broker with a different URL.\n'),
  HumanMessage(content="Edit the previous summary. Add some information about the Publish update. Start with 'SUMMARY:' followed by the edited version of the previous summary."),
  AIMessage(content="